<a href="https://colab.research.google.com/github/adiletamangossov/DataScience_portfolio/blob/main/car_damage_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    TextClassificationPipeline
)
from datasets import Dataset, DatasetDict
import numpy as np

In [ ]:
# Список категорий ремонта (ВАЖНО: порядок важен для ID)
CATEGORIES = [
    "Техническое обслуживание",
    "Двигатель",
    "Тормозная система",
    "Ходовая часть/Подвеска",
    "Электрика",
    "Трансмиссия",
    "Кузовной ремонт",
    "Диагностика",
    "Шиномонтаж",
    "Кондиционер/печка"
]

# Маппинг категорий на специалистов
CATEGORY_TO_SPECIALIST = {
    "Техническое обслуживание": "Механик ТО",
    "Двигатель": "Моторист",
    "Тормозная система": "Автослесарь (тормоза)",
    "Ходовая часть/Подвеска": "Автослесарь (подвеска)",
    "Электрика": "Автоэлектрик",
    "Трансмиссия": "Специалист по трансмиссии",
    "Кузовной ремонт": "Кузовщик / Маляр",
    "Диагностика": "Диагност",
    "Шиномонтаж": "Шиномонтажник",
    "Кондиционер/печка": "Автоэлектрик"
}

# Создаем маппинги ID <-> Название Категории
label2id = {label: i for i, label in enumerate(CATEGORIES)}
id2label = {i: label for i, label in enumerate(CATEGORIES)}

# Параметры
MODEL_NAME = "cointegrated/rubert-tiny2"
OUTPUT_DIR = "./repair_classifier_model"
TRAIN_BATCH_SIZE = 16
EVAL_BATCH_SIZE = 32
NUM_EPOCHS = 5 # Количество эпох обучения
LEARNING_RATE = 2e-5
TEST_SIZE = 0.3 # Доля данных для теста

In [ ]:
#колонка с текстом (например, 'description'), колонка с категорией (например, 'category')

data = {
    'description': [
        "Нужно поменять масло и фильтры, скоро ТО",
        "Машина не заводится, стартер крутит вхолостую",
        "При торможении слышен сильный скрип спереди",
        "Что-то стучит в подвеске при проезде неровностей",
        "Перестали гореть фары ближнего света",
        "АКПП дергается при переключении со 2 на 3 передачу",
        "Поцарапал дверь на парковке, нужна покраска",
        "Горит чек энджин, нужно понять причину",
        "Поменять летнюю резину на зимнюю",
        "Плановое ТО 60000 км",
        "Двигатель троит на холостых оборотах",
        "Замена тормозных колодок и дисков по кругу",
        "Заменить амортизаторы, машина стала жесткой",
        "Не работает кондиционер, дует теплым",
        "Рывки при разгоне, возможно сцепление",
        "Выправить вмятину на крыле без покраски",
        "Сделать компьютерную диагностику всех систем",
        "Балансировка колес, бьет руль на скорости",
        "При торможении слышен скрип, тормоза срабатывают с задержкой",
        "Машину ведёт в сторону при торможении",
        "Стук в подвеске при проезде лежачих полицейских",
        "Авто начало раскачиваться на скорости, неустойчивая подвеска",
        "Не горят стоп-сигналы, возможно перегорела лампа",
        "Перестала работать панель приборов, всё потемнело",
        "Двигатель троит на холодную, обороты нестабильны",
        "Появился посторонний шум под капотом, снижается тяга",
        "Не включается задняя передача, слышен хруст",
        "Пробуксовка при переключении передач, запах горелого масла",
        "Из воздуховодов дует только горячий воздух, не работает кондиционер",
        "Кондиционер не охлаждает, хотя включается",
        "Появилась вибрация на руле, особенно на скорости",
        "Спущено колесо, требуется замена шины"
    ],
    'category': [
        "Техническое обслуживание",
        "Двигатель",
        "Тормозная система",
        "Ходовая часть/Подвеска",
        "Электрика",
        "Трансмиссия",
        "Кузовной ремонт",
        "Диагностика",
        "Шиномонтаж",
        "Техническое обслуживание",
        "Двигатель",
        "Тормозная система",
        "Ходовая часть/Подвеска",
        "Электрика",
        "Трансмиссия",
        "Кузовной ремонт",
        "Диагностика",
        "Шиномонтаж",
        "Тормозная система",
        "Тормозная система",
        "Ходовая часть/Подвеска",
        "Ходовая часть/Подвеска",
        "Электрика",
        "Электрика",
        "Двигатель",
        "Двигатель",
        "Трансмиссия",
        "Трансмиссия",
        "Кондиционер/печка",
        "Кондиционер/печка",
        "Шиномонтаж",
        "Шиномонтаж"
    ]
}
df = pd.DataFrame(data)

# Добавляем числовые метки
df['label'] = df['category'].map(label2id)

# Проверяем, все ли категории были смаплены
if df['label'].isnull().any():
    print("ОШИБКА: Некоторые категории из данных не найдены в списке CATEGORIES!")
    # print(df[df['label'].isnull()])
    exit()

# Разделение на обучающую и тестовую выборки
train_df, test_df = train_test_split(df, test_size=TEST_SIZE, random_state=42, stratify=df['label'])

# Преобразование в формат HF Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
dataset_dict = DatasetDict({'train': train_dataset, 'test': test_dataset})

print("Пример данных:")
print(dataset_dict['train'][0])



Пример данных:
{'description': 'Горит чек энджин, нужно понять причину', 'category': 'Диагностика', 'label': 7, '__index_level_0__': 7}


In [ ]:
#Токенизация
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    # padding=True и truncation=True важны
    return tokenizer(examples["description"], padding="max_length", truncation=True, max_length=128) # max_length можно подобрать

tokenized_datasets = dataset_dict.map(tokenize_function, batched=True)

# Удаляем ненужные колонки для обучения
tokenized_datasets = tokenized_datasets.remove_columns(["description", "category", "__index_level_0__"])
tokenized_datasets.set_format("torch") # Устанавливаем формат для PyTorch

print("\nПример токенизированных данных:")
print(tokenized_datasets['train'][0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/401 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]


Пример токенизированных данных:
{'label': tensor(7), 'input_ids': tensor([    2, 51091,   751, 47053, 60785, 19706,    16, 21211, 30308, 38386,
            3,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     

In [ ]:
#Загрузка Модели
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(CATEGORIES),
    id2label=id2label,
    label2id=label2id
)

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/118M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#Fine-tuning

# Функция для вычисления метрик во время оценки
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted') # weighted важен для несбалансированных классов
    return {"accuracy": acc, "f1": f1}

# Аргументы для обучения
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=EVAL_BATCH_SIZE,
    learning_rate=LEARNING_RATE,
    weight_decay=0.01,
    evaluation_strategy="epoch", # Оценивать после каждой эпохи
    save_strategy="epoch",       # Сохранять модель после каждой эпохи
    logging_strategy="epoch",    # Логировать после каждой эпохи
    load_best_model_at_end=True, # Загрузить лучшую модель в конце обучения
    metric_for_best_model="f1",  # Использовать F1 для выбора лучшей модели
    push_to_hub=False,           # Не загружать на HF Hub
    report_to="none",            # Отключить интеграции (wandb, tensorboard)
)

# Инициализация Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Запуск обучения
print("\n--- Начало обучения ---")
trainer.train()
print("--- Обучение завершено ---")

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-22-ea548d41ba07>:29: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(



--- Начало обучения ---


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,2.303100,2.306915,0.000000,0.000000
2,2.288100,2.305175,0.000000,0.000000
3,2.293300,2.304021,0.000000,0.000000
4,2.275300,2.303222,0.000000,0.000000
5,2.263900,2.302895,0.000000,0.000000


--- Обучение завершено ---


In [ ]:
#Оценка
print("\n--- Оценка на тестовом наборе ---")
eval_results = trainer.evaluate()
print(eval_results)


--- Оценка на тестовом наборе ---


{'eval_loss': 2.306915283203125, 'eval_accuracy': 0.0, 'eval_f1': 0.0, 'eval_runtime': 0.0156, 'eval_samples_per_second': 642.697, 'eval_steps_per_second': 64.27, 'epoch': 5.0}


In [ ]:
#Сохранение модели и токенизатора
print(f"\n--- Сохранение лучшей модели в {OUTPUT_DIR} ---")
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print("--- Модель сохранена ---")


--- Сохранение лучшей модели в ./repair_classifier_model ---
--- Модель сохранена ---


In [ ]:
# Пайплайн для предсказания
# Используем модель и токенизатор из Trainer для простоты
device = 0 if torch.cuda.is_available() else -1
classifier_pipeline = TextClassificationPipeline(
    model=trainer.model,
    tokenizer=trainer.tokenizer,
    device=device
)

def classify_repair_request(text: str):
    """
    Классифицирует текстовое описание проблемы и определяет категорию и специалиста.
    """
    if not text or not text.strip():
        return {"error": "Текст заявки пуст"}

    try:
        # Получаем предсказание от модели (может вернуть список словарей)
        results = classifier_pipeline(text, return_all_scores=False) # return_all_scores=False - вернуть только лучший класс

        if not results:
             return {"error": "Модель не смогла классифицировать текст"}

        # Берем первый (и лучший) результат
        prediction = results[0]
        predicted_category = prediction['label']
        confidence = prediction['score']

        # Определяем специалиста по категории
        specialist = CATEGORY_TO_SPECIALIST.get(predicted_category, "Специалист не определен")

        return {
            "input_text": text,
            "predicted_category": predicted_category,
            "confidence": round(confidence, 4),
            "assigned_specialist": specialist
        }

    except Exception as e:
        print(f"Ошибка при классификации текста '{text}': {e}")
        return {"error": f"Внутренняя ошибка: {e}"}


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Device set to use cuda:0


In [ ]:
# --- Пример использования пайплайна ---
print("\n--- Тестирование пайплайна ---")

test_texts = [
    "Стучит спереди справа на кочках",
    "Поменять масло в двигателе и фильтр",
    "Не горит левый стоп-сигнал",
    "Нужна диагностика, горит ошибка подушки безопасности",
    "Заправить кондиционер", # Это может потребовать дообучения или более детальной категории
    "Поставить зимние шины",
    "" # Пустой текст
]

for txt in test_texts:
    result = classify_repair_request(txt)
    print(result)


--- Тестирование пайплайна ---
{'input_text': 'Стучит спереди справа на кочках', 'predicted_category': 'Тормозная система', 'confidence': 0.1155, 'assigned_specialist': 'Автослесарь (тормоза)'}
{'input_text': 'Поменять масло в двигателе и фильтр', 'predicted_category': 'Двигатель', 'confidence': 0.1073, 'assigned_specialist': 'Моторист'}
{'input_text': 'Не горит левый стоп-сигнал', 'predicted_category': 'Электрика', 'confidence': 0.1105, 'assigned_specialist': 'Автоэлектрик'}
{'input_text': 'Нужна диагностика, горит ошибка подушки безопасности', 'predicted_category': 'Тормозная система', 'confidence': 0.1083, 'assigned_specialist': 'Автослесарь (тормоза)'}
{'input_text': 'Заправить кондиционер', 'predicted_category': 'Тормозная система', 'confidence': 0.1089, 'assigned_specialist': 'Автослесарь (тормоза)'}
{'input_text': 'Поставить зимние шины', 'predicted_category': 'Тормозная система', 'confidence': 0.1046, 'assigned_specialist': 'Автослесарь (тормоза)'}
{'error': 'Текст заявки пуст

/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
